# Converting to classification problem 

Generated Bad links from graph which are not in graph and whose shortest path is greater than 2. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd
import datetime 
import time 
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams
from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import pickle
import os
#!pip install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
import random as rnd


from sklearn.model_selection import train_test_split

In [2]:
g=nx.read_edgelist('train.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(g))

indegree_dist = list(dict(g.in_degree()).values())
indegree_dist.sort()

outdegree_dist = list(dict(g.out_degree()).values())
outdegree_dist.sort()

DiGraph with 1862220 nodes and 9437519 edges


In [3]:
indegree_dist=pd.DataFrame(g.in_degree())
indegree_dist.rename(columns={1: 2},inplace=True)
indegree_dist

,0,2
0,1,3
1,690569,29
2,315892,28
3,189226,3
4,2,4
...,...,...
1862215,1510307,1
1862216,1862205,0
1862217,446775,1
1862218,1862217,0


In [4]:
out_degree=pd.DataFrame(g.out_degree())
out_degree

,0,1
0,1,3
1,690569,21
2,315892,32
3,189226,4
4,2,6
...,...,...
1862215,1510307,0
1862216,1862205,1
1862217,446775,0
1862218,1862217,3


In [5]:
out_degree.insert(2,2,indegree_dist[2])

In [6]:
out_degree

,0,1,2
0,1,3,3
1,690569,21,29
2,315892,32,28
3,189226,4,3
4,2,6,4
...,...,...,...
1862215,1510307,0,1
1862216,1862205,1,0
1862217,446775,0,1
1862218,1862217,3,0


In [7]:
sum_column = out_degree[1] + out_degree[2]
out_degree["col3"] = sum_column
out_degree

,0,1,2,col3
0,1,3,3,6
1,690569,21,29,50
2,315892,32,28,60
3,189226,4,3,7
4,2,6,4,10
...,...,...,...,...
1862215,1510307,0,1,1
1862216,1862205,1,0,1
1862217,446775,0,1,1
1862218,1862217,3,0,3


In [8]:
out_degree_sorted=out_degree.sort_values(by="col3")
out_degree_sorted

,0,1,2,col3
1862219,172329,0,1,1
1518806,521943,0,1,1
1518809,307458,0,1,1
1518810,916772,1,0,1
1518811,730399,0,1,1
...,...,...,...,...
4744,99403,264,418,682
550020,141489,759,6,765
28861,4850,895,17,912
4739,735020,652,391,1043


In [9]:
print(len(g.in_edges(735020)))
print(len(g.out_edges(735020)))

391
652


In [10]:
connections=50
nodes=1862220

In [11]:
%%time
if not os.path.isfile('Recommend.p'):
    r = csv.reader(open('train.csv','r'))
    edges = dict()
    for edge in r:
        edges[(edge[0], edge[1])] = 1
        
        
    bad_edges_set = set([])
    count=0
    while (len(bad_edges_set)< connections):
        count=count+1
        print(count)
        print(len(bad_edges_set))
        s_node=735020
        d_node=rnd.randint(1,nodes)
        tmp = edges.get((s_node,d_node),0)
        if tmp == 0 and s_node!=d_node :
            try:
                st_path=nx.shortest_path_length(g,source=s_node,target=d_node)
                if st_path==2 or st_path==3: 
                    bad_edges_set.add((s_node,d_node))
                else:
                    continue
            except:
                continue
        else:
            continue
    pickle.dump(bad_edges_set,open('Recommend.p','wb'))
else:
    bad_edges_set = pickle.load(open('Recommend.p','rb'))

1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
0
22
0
23
0
24
0
25
0
26
0
27
0
28
0
29
0
30
0
31
0
32
0
33
0
34
0
35
0
36
0
37
0
38
0
39
1
40
1
41
1
42
1
43
1
44
1
45
1
46
1
47
1
48
1
49
1
50
1
51
1
52
1
53
1
54
1
55
1
56
1
57
1
58
1
59
1
60
1
61
1
62
1
63
2
64
2
65
2
66
2
67
2
68
2
69
2
70
2
71
2
72
2
73
2
74
2
75
2
76
2
77
2
78
2
79
2
80
2
81
2
82
2
83
2
84
2
85
2
86
2
87
2
88
2
89
2
90
2
91
2
92
2
93
2
94
2
95
2
96
2
97
2
98
2
99
2
100
2
101
2
102
2
103
2
104
2
105
2
106
2
107
2
108
2
109
2
110
2
111
2
112
2
113
2
114
2
115
2
116
2
117
2
118
2
119
2
120
2
121
2
122
2
123
2
124
3
125
3
126
3
127
3
128
3
129
3
130
3
131
3
132
3
133
3
134
3
135
3
136
3
137
3
138
3
139
4
140
4
141
4
142
4
143
5
144
5
145
5
146
5
147
5
148
5
149
5
150
5
151
5
152
5
153
5
154
5
155
5
156
5
157
5
158
5
159
5
160
5
161
5
162
5
163
5
164
5
165
5
166
5
167
5
168
5
169
5
170
5
171
5
172
5
173
5
174
6
175
6
176
7
177
7
178
7
179
7
180
7
181
7
182
7
183
7
184
7
185


In [12]:
bad_edges_set = pickle.load(open('Recommend.p','rb'))
print(bad_edges_set)
len(bad_edges_set)

{(735020, 1846324), (735020, 1698702), (735020, 621115), (735020, 408522), (735020, 420335), (735020, 638349), (735020, 351358), (735020, 915318), (735020, 800685), (735020, 1180593), (735020, 214948), (735020, 1235124), (735020, 192884), (735020, 1132319), (735020, 375191), (735020, 1431431), (735020, 906827), (735020, 898437), (735020, 971752), (735020, 1535825), (735020, 919869), (735020, 644337), (735020, 730307), (735020, 110864), (735020, 330480), (735020, 967566), (735020, 770447), (735020, 474709), (735020, 630870), (735020, 1700180), (735020, 1526327), (735020, 1586441), (735020, 1007974), (735020, 180725), (735020, 471078), (735020, 571384), (735020, 435810), (735020, 1476979), (735020, 1325165), (735020, 447049), (735020, 1530939), (735020, 1382466), (735020, 203255), (735020, 291147), (735020, 1304440), (735020, 117724), (735020, 832250), (735020, 1146299), (735020, 1234249), (735020, 1429879)}


50

## Splitting positive and negative connections

In [13]:
if (not os.path.isfile('Recommend.csv')):
    Bad_edges = pd.DataFrame(list(bad_edges_set), columns=['source', 'destination'])

    y_train_neg=np.zeros(len(Bad_edges))
    
    print("Bad edges in graph = ", Bad_edges.shape[0])
    print(y_train_neg)

    #saving
    Bad_edges.to_csv('Recommend.csv',header=False, index=False)
    print(Bad_edges.head())
else:
    #Graph from Traing data only 
    print('deleting ........')
    del bad_edges_set

Bad edges in graph =  50
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
   source  destination
0  735020      1846324
1  735020      1698702
2  735020       621115
3  735020       408522
4  735020       420335


In [35]:
#final train and test data sets
#if (not os.path.isfile('Recommend.csv')):
pd.DataFrame(y_train_neg.astype(int)).to_csv('train_y_negs.csv',header=False,index=False)

# Features

In [15]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

# Reading Data

In [16]:
train_graph=nx.read_edgelist('train.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(train_graph))

DiGraph with 1862220 nodes and 9437519 edges


 # Similarity measures

## Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [17]:
#for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [18]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [19]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [20]:
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Ranking Measures

https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.


Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

## Page Ranking

https://en.wikipedia.org/wiki/PageRank



In [21]:
if not os.path.isfile('page_rank.p'):
    pr = nx.pagerank(train_graph, alpha=0.85)
    pickle.dump(pr,open('page_rank.p','wb'))
else:
    pr = pickle.load(open('page_rank.p','rb'))

In [22]:
print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))

min 1.478340267341635e-07
max 2.688578764999046e-05
mean 5.369934808940616e-07


In [23]:
#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.369934808940616e-07


## Shortest path:

In [24]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

## Checking for same community

In [25]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

## Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

X is considered the source and y the destination node, N(u) are the common neighbours of the two.

In [26]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

## Following back

In [27]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

## Katz Centrality:

In [28]:
if not os.path.isfile('katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('katz.p','wb'))
else:
    katz = pickle.load(open('katz.p','rb'))

In [29]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007101687329270317
max 0.00557733722363664
mean 0.0007306395563933362


In [30]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007306395563933362


## Hits Algorithm

In [31]:
if not os.path.isfile('hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('hits.p','wb'))
else:
    hits = pickle.load(open('hits.p','rb'))

In [32]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min -2.249912022959974e-20
max 0.004813673061976435
mean 5.369934808962772e-07


## Choosing the number of occasions that will be trained

In [33]:
import random
if os.path.isfile('Recommend.csv'):
    filename = "Recommend.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    #n_train =  15100028
    s = n_train # HOW MANY TO READ
    skip_train = 0#sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

# From Sample to Full

In [36]:
df_final_train = pd.read_csv('Recommend.csv', skiprows=skip_train, names=['source', 'destination'])
#df_final_train = pd.read_csv('Recommend.csv', names=['source', 'destination'])
df_final_train['connected'] = pd.read_csv('train_y_negs.csv', skiprows=skip_train, names=['connected'])
#df_final_train['connected'] = pd.read_csv('train_y_negs.csv', names=['connected'])

print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (50, 3)


,source,destination,connected
0,735020,1846324,0
1,735020,1698702,0


## Adding features as collumns


In [37]:
if not os.path.isfile('Recommend_sample_stage1.h5'):
    #mapping jaccrd followers to train and test data
    df_final_train['jaccard_followers'] = df_final_train.apply(lambda row:
                                            jaccard_for_followers(row['source'],row['destination']),axis=1)


    #mapping jaccrd followees to train and test data
    df_final_train['jaccard_followees'] = df_final_train.apply(lambda row:
                                            jaccard_for_followees(row['source'],row['destination']),axis=1)

        #mapping jaccrd followers to train and test data
    df_final_train['cosine_followers'] = df_final_train.apply(lambda row:
                                            cosine_for_followers(row['source'],row['destination']),axis=1)

    #mapping jaccrd followees to train and test data
    df_final_train['cosine_followees'] = df_final_train.apply(lambda row:
                                            cosine_for_followees(row['source'],row['destination']),axis=1)

In [38]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source']))
            s2=set(train_graph.successors(row['source']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination']))
            d2=set(train_graph.successors(row['destination']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return num_followers_s, num_followers_d, num_followees_s, num_followees_d, inter_followers, inter_followees

In [39]:
if not os.path.isfile('Recommend_sample_stage1.h5'):
    df_final_train['num_followers_s'], df_final_train['num_followers_d'], \
    df_final_train['num_followees_s'], df_final_train['num_followees_d'], \
    df_final_train['inter_followers'], df_final_train['inter_followees']= compute_features_stage1(df_final_train)
    

    hdf = HDFStore('Recommend_sample_stage1.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('Recommend_sample_stage1.h5', 'train_df',mode='r')

## Stage 2

In [40]:
if not os.path.isfile('Recommend_sample_stage2.h5'):
    #mapping adar index on train
    df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source'],row['destination']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping followback or not on train
    df_final_train['follows_back'] = df_final_train.apply(lambda row: follows_back(row['source'],row['destination']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping same component of wcc or not on train
    df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source'],row['destination']),axis=1)
    
#    --------------------------------------------------------------------------------------------------------
    #mapping shortest path on train 
    df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source'],row['destination']),axis=1)
    #mapping shortest path on test

    hdf = HDFStore('Recommend_sample_stage2.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('Recommend_sample_stage2.h5', 'train_df',mode='r')

## Stage 3


<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>


#### Weight Features

In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 
`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

it is directed graph so calculated Weighted in and Weighted out differently

In [41]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|█████████████████████████████████████████████████████████████████████| 1862220/1862220 [00:28<00:00, 66181.60it/s]


In [42]:
if not os.path.isfile('Recommend_sample_stage3.h5'):
    #mapping to pandas train
    df_final_train['weight_in'] = df_final_train.destination.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_train['weight_out'] = df_final_train.source.apply(lambda x: Weight_out.get(x,mean_weight_out))


    #some features engineerings on the in and out weights
    df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
    df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
    df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
    df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)


In [43]:
if not os.path.isfile('Recommend_sample_stage3.h5'):
    
    #page rank 
    #if anything not there in train graph then adding mean page rank 
    df_final_train['page_rank_s'] = df_final_train.source.apply(lambda x:pr.get(x,mean_pr))
    df_final_train['page_rank_d'] = df_final_train.destination.apply(lambda x:pr.get(x,mean_pr))


    #================================================================================

    #Katz centrality score 
    #if anything not there in train graph then adding mean katz score
    df_final_train['katz_s'] = df_final_train.source.apply(lambda x: katz.get(x,mean_katz))
    df_final_train['katz_d'] = df_final_train.destination.apply(lambda x: katz.get(x,mean_katz))


    #================================================================================

    #Hits algorithm score 
    #if anything not there in train graph then adding 0
    df_final_train['hubs_s'] = df_final_train.source.apply(lambda x: hits[0].get(x,0))
    df_final_train['hubs_d'] = df_final_train.destination.apply(lambda x: hits[0].get(x,0))

    #================================================================================

    #Hits algorithm score 
    #if anything not there in train graph then adding 0
    df_final_train['authorities_s'] = df_final_train.source.apply(lambda x: hits[1].get(x,0))
    df_final_train['authorities_d'] = df_final_train.destination.apply(lambda x: hits[1].get(x,0))


    #================================================================================

    hdf = HDFStore('Recommend_sample_stage3.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('Recommend_sample_stage3.h5', 'train_df',mode='r')

In [44]:
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,735020,1846324,0,0.0,0.000000,0.0,0.000000,391,10,652,...,0.642156,0.379777,0.000006,5.642064e-07,0.002674,0.000750,4.269629e-12,3.114232e-14,2.924335e-12,6.938837e-15
1,735020,1698702,0,0.0,0.000000,0.0,0.000000,391,20,652,...,0.475569,0.296484,0.000006,2.092265e-06,0.002674,0.000785,4.269629e-12,2.960183e-15,2.924335e-12,4.375862e-15
2,735020,621115,0,0.0,0.000000,0.0,0.000000,391,1,652,...,1.453347,0.785373,0.000006,1.520216e-07,0.002674,0.000714,4.269629e-12,3.965015e-16,2.924335e-12,1.829239e-15
3,735020,408522,0,0.0,0.002920,0.0,0.013240,391,35,652,...,0.372466,0.244933,0.000006,2.107550e-06,0.002674,0.000853,4.269629e-12,3.051558e-15,2.924335e-12,1.387670e-14
4,735020,420335,0,0.0,0.001531,0.0,0.027692,391,5,652,...,0.855630,0.486514,0.000006,4.014519e-07,0.002674,0.000730,4.269629e-12,2.616759e-16,2.924335e-12,3.640880e-15


### Adding new feature Preferential Attachement 

 One well-known concept in social networks is that users with many friends tend to create more connections in the future. This is due to the fact that in some social networks, like in finance, the rich get richer. We estimate how ”rich” our two vertices are by calculating the multiplication between the number of friends (|Γ(x)|) or followers each vertex has.

#### Preferential Attachement for followers

In [45]:
#for train dataset
nfs=np.array(df_final_train['num_followers_s'])
nfd=np.array(df_final_train['num_followers_d'])
preferential_followers=[]
for i in range(len(nfs)):
    preferential_followers.append(nfd[i]*nfs[i])
df_final_train['prefer_Attach_followers']= preferential_followers
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,prefer_Attach_followers
0,735020,1846324,0,0.0,0.000000,0.0,0.000000,391,10,652,...,0.379777,0.000006,5.642064e-07,0.002674,0.000750,4.269629e-12,3.114232e-14,2.924335e-12,6.938837e-15,3910
1,735020,1698702,0,0.0,0.000000,0.0,0.000000,391,20,652,...,0.296484,0.000006,2.092265e-06,0.002674,0.000785,4.269629e-12,2.960183e-15,2.924335e-12,4.375862e-15,7820
2,735020,621115,0,0.0,0.000000,0.0,0.000000,391,1,652,...,0.785373,0.000006,1.520216e-07,0.002674,0.000714,4.269629e-12,3.965015e-16,2.924335e-12,1.829239e-15,391
3,735020,408522,0,0.0,0.002920,0.0,0.013240,391,35,652,...,0.244933,0.000006,2.107550e-06,0.002674,0.000853,4.269629e-12,3.051558e-15,2.924335e-12,1.387670e-14,13685
4,735020,420335,0,0.0,0.001531,0.0,0.027692,391,5,652,...,0.486514,0.000006,4.014519e-07,0.002674,0.000730,4.269629e-12,2.616759e-16,2.924335e-12,3.640880e-15,1955


#### Preferential Attachement for followees

In [46]:
#for train dataset
nfs=np.array(df_final_train['num_followees_s'])
nfd=np.array(df_final_train['num_followees_d'])
preferential_followees=[]
for i in range(len(nfs)):
    preferential_followees.append(nfd[i]*nfs[i])
df_final_train['prefer_Attach_followees']= preferential_followees
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,prefer_Attach_followers,prefer_Attach_followees
0,735020,1846324,0,0.0,0.000000,0.0,0.000000,391,10,652,...,0.000006,5.642064e-07,0.002674,0.000750,4.269629e-12,3.114232e-14,2.924335e-12,6.938837e-15,3910,5216
1,735020,1698702,0,0.0,0.000000,0.0,0.000000,391,20,652,...,0.000006,2.092265e-06,0.002674,0.000785,4.269629e-12,2.960183e-15,2.924335e-12,4.375862e-15,7820,11084
2,735020,621115,0,0.0,0.000000,0.0,0.000000,391,1,652,...,0.000006,1.520216e-07,0.002674,0.000714,4.269629e-12,3.965015e-16,2.924335e-12,1.829239e-15,391,652
3,735020,408522,0,0.0,0.002920,0.0,0.013240,391,35,652,...,0.000006,2.107550e-06,0.002674,0.000853,4.269629e-12,3.051558e-15,2.924335e-12,1.387670e-14,13685,22820
4,735020,420335,0,0.0,0.001531,0.0,0.027692,391,5,652,...,0.000006,4.014519e-07,0.002674,0.000730,4.269629e-12,2.616759e-16,2.924335e-12,3.640880e-15,1955,1304


In [47]:

hdf = HDFStore('Recommend_sample_stage4.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.close()

In [48]:
df_final_train.groupby(by="connected")

In [49]:
df_final_train["connected"].value_counts()

0    50
Name: connected, dtype: int64

In [51]:
g.edges(735020)

OutEdgeDataView([(735020, 2022), (735020, 1353883), (735020, 826738), (735020, 922231), (735020, 1670816), (735020, 912429), (735020, 1775806), (735020, 1472169), (735020, 1347809), (735020, 4850), (735020, 162042), (735020, 212296), (735020, 82582), (735020, 354781), (735020, 1731476), (735020, 344947), (735020, 1274972), (735020, 81865), (735020, 915809), (735020, 1418350), (735020, 17422), (735020, 1545474), (735020, 1613333), (735020, 1675147), (735020, 431481), (735020, 958403), (735020, 1525064), (735020, 431238), (735020, 356403), (735020, 1708179), (735020, 1757017), (735020, 1526989), (735020, 667716), (735020, 1428077), (735020, 49726), (735020, 741911), (735020, 1270076), (735020, 1385047), (735020, 661227), (735020, 1457619), (735020, 99403), (735020, 23585), (735020, 1065392), (735020, 373599), (735020, 1199655), (735020, 129694), (735020, 121355), (735020, 596112), (735020, 1667424), (735020, 283895), (735020, 565760), (735020, 332962), (735020, 1626466), (735020, 1562431